In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import xmltodict, json
import matplotlib.pyplot as plt
import os
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import textwrap

In [2]:
#directory 
directory = os.getcwd()
save_loc = f'{directory}/generated_gif/'

def get_data(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'xml')
    data = xmltodict.parse(str(soup))
    return data

def get_obs_data(dataframe):
    x_dict = []
    y_dict = []
    for i in range(0,len(dataframe['Obs'])):
        x = dataframe['Obs'][i]['@TIME_PERIOD'] 
        y = dataframe['Obs'][i]['@OBS_VALUE']
        x = pd.to_datetime(x) #convert x to datetime
        try:
            y = float(y)
        except:
            y = np.nan
        x_dict.append(x)
        y_dict.append(y)
    return x_dict, y_dict

# gif with multiple indicators
def make_gif_multivar(plotname, df, shorten=True):
    plotname = str(plotname)
    filenames = []
    labels = []
    ys = {}
    xs = {}
    x = get_obs_data(df[0])[0]
    for i in range(0, len(df)):
        label_raw = desc[desc['indicator'] == df[i]['@INDICATOR']]['desc'].values[0]
        if shorten:
            label = textwrap.fill(label_raw, 15)
        else:
            label = label_raw
        labels.append(label)
        xs[label], ys[label] = get_obs_data(df[i])
    for time in range(0, len(x)):
        fig, ax = plt.subplots(figsize=(10,5))
        for j in range(0, len(labels)):
            plt.plot(xs[labels[j]][:time], ys[labels[j]][:time], label=labels[j])
        plt.xticks(rotation=45, size=10)
        plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
        #plt.tight_layout()
        #create file name and add to list
        filename = f'{time}.png'
        filenames.append(filename)
        #save frames
        plt.title(plotname)
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
    # build gif
    gif_filename = f'{save_loc}{plotname}.gif'
    images = []
    duration = 0.15

    with imageio.get_writer(gif_filename, mode='I', duration=duration, loop=1) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Remove files
    for filename in set(filenames):
        os.remove(filename)


In [38]:
#database with url
database = pd.DataFrame(pd.read_csv('https://raw.githubusercontent.com/thanhqtran/gso-macro-monitor/main/dsbb_database.csv')).reset_index(drop=True)
#convert data values to string 
database['url'].apply(lambda x: str(x))
database['var'].apply(lambda x: str(x))

#database description
desc = pd.DataFrame(pd.read_csv('https://raw.githubusercontent.com/thanhqtran/gso-macro-monitor/main/dsbb_indicator_desc.csv')).reset_index(drop=True)
desc['indicator'].apply(lambda x: str(x))
desc['desc'].apply(lambda x: str(x))
desc['domain'].apply(lambda x: str(x))
# do not display output 
pd.set_option('display.max_columns', None)

In [22]:
df_gdp = get_data(database['url'][database['var'] == 'gdp'].values[0])
df_cpi = get_data(database['url'][database['var'] == 'cpi'].values[0])
df_govbudget = get_data(database['url'][database['var'] == 'gov_budget'].values[0])
df_gov_debt = get_data(database['url'][database['var'] == 'gov_debt'].values[0])
df_interest = get_data(database['url'][database['var'] == 'interest'].values[0])
df_stock = get_data(database['url'][database['var'] == 'stock'].values[0])
df_bop = get_data(database['url'][database['var'] == 'bop'].values[0])
df_extdebt = get_data(database['url'][database['var'] == 'ext_debt'].values[0])
df_exr = get_data(database['url'][database['var'] == 'exchange_rt'].values[0])
df_iip = get_data(database['url'][database['var'] == 'prod_index'].values[0])
df_lab = get_data(database['url'][database['var'] == 'labor'].values[0])
df_ppi = get_data(database['url'][database['var'] == 'ppi'].values[0])
df_pop = get_data(database['url'][database['var'] == 'pop'].values[0])
df_trade = get_data(database['url'][database['var'] == 'trade'].values[0])

In [10]:
### GDP data
df_gdp1 = df_gdp['message:StructureSpecificData']['message:DataSet']['Series']
# extract data based on INDICATOR
agri_inds = ['NGDPVA_R_ISIC4_A01_XDC', 'NGDPVA_R_ISIC4_A02_XDC', 'NGDPVA_R_ISIC4_A03_XDC']
industrial_inds = ['NGDPVA_R_ISIC4_B_XDC','NGDPVA_R_ISIC4_C_XDC','NGDPVA_R_ISIC4_D_XDC','NGDPVA_R_ISIC4_E_XDC','NGDPVA_R_ISIC4_F_XDC']
service_inds = ['NGDPVA_R_ISIC4_G_XDC','NGDPVA_R_ISIC4_H_XDC','NGDPVA_R_ISIC4_I_XDC','NGDPVA_R_ISIC4_J_XDC','NGDPVA_R_ISIC4_K_XDC','NGDPVA_R_ISIC4_L_XDC','NGDPVA_R_ISIC4_M_XDC','NGDPVA_R_ISIC4_N_XDC','NGDPVA_R_ISIC4_O_XDC','NGDPVA_R_ISIC4_P_XDC','NGDPVA_R_ISIC4_Q_XDC','NGDPVA_R_ISIC4_R_XDC','NGDPVA_R_ISIC4_S_XDC']
extracted_agri = []
extracted_inds = []
extracted_serv = []
#loop through the list and extract matched indicators
for item in df_gdp1:
    if item['@INDICATOR'] in agri_inds and item['@FREQ'] == 'Q':
        extracted_agri.append(item)
    elif item['@INDICATOR'] in industrial_inds and item['@FREQ'] == 'Q':
        extracted_inds.append(item)
    elif item['@INDICATOR'] in service_inds and item['@FREQ'] == 'Q':
        extracted_serv.append(item)
    else:
        pass

# make gif based on INDICATOR
make_gif_multivar('Real GDP Agriculture', extracted_agri)
make_gif_multivar('Real GDP Construction and Industry', extracted_inds)
make_gif_multivar('Real GDP Services', extracted_serv)

/var/folders/00/9_g7krb12h16y6njhqkpps_h0000gn/T/ipykernel_30843/486125819.py:63: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [20]:
### Labor data 
df_lab = get_data(database['url'][database['var'] == 'labor'].values[0])
df_lab1 = df_lab['message:StructureSpecificData']['message:DataSet']['Series']
unempl_inds = ['LUR_PT','LUR15O_PT','LEU_PT']
empl_inds = ['LLF_PE_NUM','LE_PE_NUM']
extracted_unempl = []
extracted_empl = []
for item in df_lab1:
    if item['@INDICATOR'] in unempl_inds and item['@FREQ'] == 'Q':
        extracted_unempl.append(item)
    elif item['@INDICATOR'] in empl_inds and item['@FREQ'] == 'Q':
        extracted_empl.append(item)
    else:
        pass
extracted_unempl
#make_gif_multivar('Unemployment', extracted_unempl, shorten=True)
#make_gif_multivar('Employment', extracted_empl, shorten=True)

[{'@COUNTERPART_AREA': '_Z',
  '@DATA_DOMAIN': 'LMI',
  '@FREQ': 'Q',
  '@INDICATOR': 'LUR_PT',
  '@REF_AREA': 'VN',
  '@UNIT_MULT': '3',
  'Obs': [{'@OBS_VALUE': '2.25', '@TIME_PERIOD': '2016-Q1'},
   {'@OBS_VALUE': '2.29', '@TIME_PERIOD': '2016-Q2'},
   {'@OBS_VALUE': '2.34', '@TIME_PERIOD': '2016-Q3'},
   {'@OBS_VALUE': '2.31', '@TIME_PERIOD': '2016-Q4'},
   {'@OBS_VALUE': '2.3', '@TIME_PERIOD': '2017-Q1'},
   {'@OBS_VALUE': '2.26', '@TIME_PERIOD': '2017-Q2'},
   {'@OBS_VALUE': '2.23', '@TIME_PERIOD': '2017-Q3'},
   {'@OBS_VALUE': '2.21', '@TIME_PERIOD': '2017-Q4'},
   {'@OBS_VALUE': '2.2', '@TIME_PERIOD': '2018-Q1'},
   {'@OBS_VALUE': '2.19', '@TIME_PERIOD': '2018-Q2'},
   {'@OBS_VALUE': '2.2', '@TIME_PERIOD': '2018-Q3'},
   {'@OBS_VALUE': '2.2', '@TIME_PERIOD': '2018-Q4'},
   {'@OBS_VALUE': '2.17', '@TIME_PERIOD': '2019-Q1'},
   {'@OBS_VALUE': '2.16', '@TIME_PERIOD': '2019-Q2'},
   {'@OBS_VALUE': '2.17', '@TIME_PERIOD': '2019-Q3'},
   {'@OBS_VALUE': '2.15', '@TIME_PERIOD': '2019-Q

In [67]:
### Earnings
earning_inds = ['LME_AVG_XDC','LMEM_AVG_XDC','LMEF_AVG_XDC']
extracted_earning = []
for item in df_lab1:
    if item['@INDICATOR'] in earning_inds and item['@FREQ'] == 'Q':
        extracted_earning.append(item)
    else:
        pass
make_gif_multivar('Earnings', extracted_earning, shorten=True)

In [59]:
### Exchange rate data
df_exr1 = df_exr['message:StructureSpecificData']['message:DataSet']['Series']
exchange_rate = ['ENDE_MID_XDC_USD_RATE', 'ENDA_MID_XDC_USD_RATE']
extracted_exr = []
for item in df_exr1:
    if item['@INDICATOR'] in exchange_rate and item['@FREQ'] == 'M':
        extracted_exr.append(item)
    else:
        pass
make_gif_multivar('Exchange Rate', extracted_exr, shorten=True)

In [27]:
### BOP 
df_bop1 = df_bop['message:StructureSpecificData']['message:DataSet']['Series']
bop_inds = ['VNM_BCA_BP6_USD','VNM_BK_BP6_USD','VNM_BF_BP6_USD']
inflow_inds = ['VNM_BFDL_BP6_USD','VNM_BFPL_BP6_USD','VNM_BFOL_BP6_USD']
outflow_inds = ['VNM_BFDA_BP6_USD','VNM_BFPA_BP6_USD','VNM_BFOA_BP6_USD']
extracted_bop = []
extracted_invest = []
for item in df_bop1:
    if item['@INDICATOR'] in bop_inds and item['@FREQ'] == 'Q':
        extracted_bop.append(item)
    elif item['@INDICATOR'] in inflow_inds and item['@FREQ'] == 'Q':
        extracted_invest.append(item)
    else:
        pass
make_gif_multivar('BOP', extracted_bop, shorten=True)
make_gif_multivar('Investment', extracted_invest, shorten=True)

In [61]:
### Producer Price Index 
df_ppi1 = df_ppi['message:StructureSpecificData']['message:DataSet']['Series']
ppi_inds = ['PPPI_ISIC4_A_BY_PP_IX','VNM_PPPI_ISIC4_BTE_BY_PP_IX','VNM_PPPI_ISIC4_HTJ_PTR_BY_PP_IX']
extracted_ppi = []
for item in df_ppi1:
    if item['@INDICATOR'] in ppi_inds and item['@FREQ'] == 'Q':
        extracted_ppi.append(item)
    else:
        pass
make_gif_multivar('Producer Price Index', extracted_ppi, shorten=True)

In [41]:
### CPI 
df_cpi1 = df_cpi['message:StructureSpecificData']['message:DataSet']['Series']
## decomposed CPI
cpi_comp_inds = ['PCPI_CP_01_IX','PCPI_CP_02_IX','PCPI_CP_03_IX','PCPI_CP_04_IX','PCPI_CP_05_IX','PCPI_CP_06_IX','PCPI_CP_07_IX','PCPI_CP_08_IX','PCPI_CP_10_IX','PCPI_CP_09_IX','PCPI_CP_12_IX']
## aggregate CPI
cpi_agg_inds = ['PCPI_IX', 'PCPICO_PC_PP_PT']

## extract each item to a list
extracted_cpi_comp = []
extracted_cpi_agg = []
for item in df_cpi1:
    if item['@INDICATOR'] in cpi_comp_inds and item['@FREQ'] == 'M':
        extracted_cpi_comp.append(item)
    elif item['@INDICATOR'] in cpi_agg_inds and item['@FREQ'] == 'M':
        extracted_cpi_agg.append(item)
    else:
        pass
#make_gif_multivar('CPI Components', extracted_cpi_comp, shorten=True)

# Main CPI index and core CPI index
cpi_agg = df_cpi1[0]
cpi_core = df_cpi1[32]
agg_x, agg_y = get_obs_data(cpi_agg)
core_x, core_y = get_obs_data(cpi_core)
label_agg = desc[desc['indicator'] == cpi_agg['@INDICATOR']]['desc'].values[0]
label_core = desc[desc['indicator'] == cpi_core['@INDICATOR']]['desc'].values[0]

# plot 2 graphs in 1 (different y scale)
filenames = []
for i in range(0, len(agg_x)):
    fig, ax1 = plt.subplots(figsize=(10,5))
    ax1.plot(agg_x[:i], agg_y[:i], label=label_agg, color='b', marker='o')
    ax1.set_ylabel(label_agg, color='b')
    ax1.tick_params(axis='x', labelrotation=90)
    ax2 = ax1.twinx()
    ax2.plot(core_x[:i], core_y[:i], label=label_core, color='r', marker='o')
    ax2.set_ylabel(label_core, color='r')
    fig.tight_layout()
    #create file name and add to list
    filename = f'{i}.png'
    filenames.append(filename)
    #save frames
    plt.title('Vietnam CPI Index and Core CPI change yoy')
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
# build gif
gif_filename = f'{save_loc}cpi.gif'
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
duration = 0.15
imageio.mimsave(gif_filename, images, duration=duration, loop=1)
# Remove files
for filename in set(filenames):
    os.remove(filename)

/var/folders/00/9_g7krb12h16y6njhqkpps_h0000gn/T/ipykernel_30843/3834760603.py:50: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))


In [42]:
### Population
df_pop1 = df_pop['message:StructureSpecificData']['message:DataSet']['Series']
pop_inds = ['LPM_PE_NUM','LPF_PE_NUM']
urban_inds = ['LPU_PE_NUM','LPR_PE_NUM']
extracted_pop = []
extracted_urban = []
for item in df_pop1:
    if item['@INDICATOR'] in pop_inds and item['@FREQ'] == 'A':
        extracted_pop.append(item)
    elif item['@INDICATOR'] in urban_inds and item['@FREQ'] == 'A':
        extracted_urban.append(item)
    else:
        pass
make_gif_multivar('Population', extracted_pop, shorten=True)

/var/folders/00/9_g7krb12h16y6njhqkpps_h0000gn/T/ipykernel_30843/486125819.py:63: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [43]:
### Trade 
df_trade1 = df_trade['message:StructureSpecificData']['message:DataSet']['Series']
trade_inds = ['TXG_FOB_USD','TMG_CIF_USD']
export_inds = ['TXG_FOB_USD', 'TXGBD_FOB_USD', 'VNM_TXG_FS_FOB_USD']
import_inds = ['TMG_CIF_USD', 'VNM_TMG_DS_CIF_USD', 'VNM_TMG_FS_CIF_USD']
extracted_trade = []
extracted_export = []
extracted_import = []
for item in df_trade1:
    if item['@INDICATOR'] in trade_inds and item['@FREQ'] == 'M':
        extracted_trade.append(item)
    elif item['@INDICATOR'] in export_inds and item['@FREQ'] == 'M':
        extracted_export.append(item)
    elif item['@INDICATOR'] in import_inds and item['@FREQ'] == 'M':
        extracted_import.append(item)
    else:
        pass
make_gif_multivar('Trade', extracted_trade, shorten=True)
make_gif_multivar('Export', extracted_export, shorten=True)
make_gif_multivar('Import', extracted_import, shorten=True)

/var/folders/00/9_g7krb12h16y6njhqkpps_h0000gn/T/ipykernel_30843/486125819.py:63: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


IndexError: list index out of range

In [44]:
df_trade1


[{'@COUNTERPART_AREA': '_Z',
  '@DATA_DOMAIN': 'MET',
  '@FREQ': 'M',
  '@INDICATOR': 'TB_USD',
  '@REF_AREA': 'VN',
  '@UNIT_MULT': '6',
  'Obs': [{'@OBS_VALUE': '-981.1916659999997', '@TIME_PERIOD': '2010-01'},
   {'@OBS_VALUE': '-1395.9445850000002', '@TIME_PERIOD': '2010-02'},
   {'@OBS_VALUE': '-1214.1656990000001', '@TIME_PERIOD': '2010-03'},
   {'@OBS_VALUE': '-1220.8189030000003', '@TIME_PERIOD': '2010-04'},
   {'@OBS_VALUE': '-893.5880790000001', '@TIME_PERIOD': '2010-05'},
   {'@OBS_VALUE': '-751.6420199999993', '@TIME_PERIOD': '2010-06'},
   {'@OBS_VALUE': '-1031.472307', '@TIME_PERIOD': '2010-07'},
   {'@OBS_VALUE': '-484.82314899999983', '@TIME_PERIOD': '2010-08'},
   {'@OBS_VALUE': '-882.6119370000006', '@TIME_PERIOD': '2010-09'},
   {'@OBS_VALUE': '-1113.9493659999998', '@TIME_PERIOD': '2010-10'},
   {'@OBS_VALUE': '-1346.2904559999997', '@TIME_PERIOD': '2010-11'},
   {'@OBS_VALUE': '-1285.7981170000003', '@TIME_PERIOD': '2010-12'},
   {'@OBS_VALUE': '-863.6385950000004'

In [25]:
### Stock market
df_stock1 = df_stock['message:StructureSpecificData']['message:DataSet']['Series']
stock_inds = ['VNM_VN_EOP_IX','VNM_HNX_EOP_IX','VNM_VNX_EOP_IX']
stock30_inds = ['VNM_VN30_EOP_IX','VNM_HNX30_EOP_IX']
extracted_stock = []
extracted_stock30 = []
for item in df_stock1:
    if item['@INDICATOR'] in stock_inds and item['@FREQ'] == 'M':
        extracted_stock.append(item)
    elif item['@INDICATOR'] in stock30_inds and item['@FREQ'] == 'M':
        extracted_stock30.append(item)
    else:
        pass
make_gif_multivar('Stock Main Indices', extracted_stock, shorten=True)
make_gif_multivar('Stock Main Indices Top 30', extracted_stock30, shorten=True)